In [57]:
%matplotlib inline

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [58]:
%store -r daily_data_df

In [59]:
data_df = daily_data_df

In [60]:
df = data_df.loc['2017-01-01':'2020-01-01'].dropna()
data = df['Nord Pool Lietuva']

In [81]:
class markov_autoregression_wrapper:
    def __init__(self, model, data):
        self.model = model
        self.data = data

    def fit_model(self):
        self.model_result = self.model.fit()

    def print_summary(self):
        display(self.model_result.summary())


    def show_probabilities_chart(self):
        fig, axes = plt.subplots(3, figsize=(24,10))

        ax = axes[0]
        ax.plot(self.model_result.smoothed_marginal_probabilities[0])
        ax.set(title='Soothed probability of regime 0')

        ax = axes[1]
        ax.plot(self.model_result.filtered_marginal_probabilities[0])
        ax.set(title='Filtered probability of regime 0')

        ax = axes[2]
        ax.plot(self.data)
        ax.set(title='Original time series')

        display(fig.tight_layout())


    def in_place_prediction(self):
        if not hasattr(self, 'prediction_result') or self.prediction_result is None:
            self.prediction_result = self.model_result.predict()
        return self.prediction_result

    def show_prediction_chart(self):
        result = self.in_place_prediction()

        data_index = pd.DatetimeIndex(self.data.index.to_timestamp())
        prediction_index = pd.DatetimeIndex(result.index.to_timestamp())
        price_data_original = go.Scatter(x=data_index,
                         y=self.data,
                         name="Original time series")
        price_data_prediction = go.Scatter(x=prediction_index,
                                y=result,
                                name="In place prediction")
        layout = go.Layout(title='Energy Plot', xaxis=dict(title='Laikas'),
                   yaxis=dict(title='(Elektros kaina EUR/MWh)'),
                   height=400, width=1400)
        fig = go.Figure(data=[price_data_original, price_data_prediction], layout=layout)
        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))
        display(fig.show())

    def calculate_errors(self):
        result = self.in_place_prediction()

        df = pd.concat([self.data, result], axis = 1, keys=["expected", "predictions"]) 
        df = df.dropna()

        expected = df['expected']
        predictions = df['predictions']

        mae = mean_absolute_error(expected, predictions)
        mse = mean_squared_error(expected, predictions)
        rmse = math.sqrt(mse)

        aic = self.model_result.aic
        bic = self.model_result.bic
        hqic = self.model_result.hqic
        
        print('MAE = {}\nMSE = {}\nRMSE = {}\nAIC = {}\nBIC = {}\nHQIC = {}'
            .format(mae, mse, rmse, aic, bic, hqic))

In [88]:
df.columns

Index(['Prognozuojama vėjo elektrinių gamyba',
       'Vėjo elektrinių gamybos planas', 'Faktinė vėjo elektrinių gamyba',
       'Planuojama nacionalinė elektros energijos gamyba',
       'Faktinė nacionalinė elektros energijos gamyba',
       'Prognozuojamas nacionalinis elektros energijos suvartojimas',
       'Planuojamas nacionalinis elektros energijos suvartojimas',
       'Faktinis nacionalinis Elektros energijos vartojimas',
       'Nord Pool Lietuva'],
      dtype='object')

In [92]:
exog_cols = ['Faktinė vėjo elektrinių gamyba', 'Faktinė nacionalinė elektros energijos gamyba', 'Faktinis nacionalinis Elektros energijos vartojimas']

In [100]:
exog_cols = ['Faktinė vėjo elektrinių gamyba']

In [109]:
exog_matrix = np.array(df[exog_cols].to_numpy())

In [110]:
mod_hamilton = sm.tsa.MarkovAutoregression(data, k_regimes=2, order=2, switching_ar=True, switching_variance=False, exog=exog_matrix, switching_exog=True)
wrapper = markov_autoregression_wrapper(mod_hamilton, data)
wrapper.fit_model()
wrapper.print_summary()
wrapper.show_prediction_chart()
wrapper.calculate_errors()

/home/majo/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



<class 'statsmodels.iolib.summary.Summary'>
"""
                         Markov Switching Model Results                         
================================================================================
Dep. Variable:        Nord Pool Lietuva   No. Observations:                 1088
Model:             MarkovAutoregression   Log Likelihood               -3741.955
Date:                  Sun, 14 Mar 2021   AIC                           7505.910
Time:                          15:06:29   BIC                           7560.823
Sample:                      01-01-2017   HQIC                          7526.695
                           - 01-01-2020                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.6090      4.212      2.519      0.012       2.354      18.864
x1             0.0971      0.027      3.549      0.000       0.043       0.151
ar.L1         -0.0505      0.297     -0.170      0.865      -0.633       0.532
ar.L2         -0.0412      1.036     -0.040      0.968      -2.073       1.990
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.5766      0.914     49.871      0.000      43.785      47.368
x1            -0.0120      0.000    -60.750      0.000      -0.012      -0.012
ar.L1          0.6952      0.033     21.347      0.000       0.631       0.759
ar.L2          0.0614      0.030      2.027      0.043       0.002       0.121
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        54.9882      3.090     17.794      0.000      48.931      61.045
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.0655      0.316      0.207      0.836      -0.553       0.685
p[1->0]        0.0115      0.004      2.874      0.004       0.004       0.019
==============================================================================

Warnings:
[1] Covariance matrix calculated using numerical (complex-step) differentiation.
"""

None

MAE = 5.223686935753992
MSE = 49.381059450082226
RMSE = 7.02716581916794
AIC = 7505.910161514135
BIC = 7560.82322221571
HQIC = 7526.695330960155
